In [1]:
import os
gpu_num = 0 # 使用 "" 来启用 CPU
os.environ["CUDA_VISIBLE_DEVICES"] = f"{gpu_num}"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)
# Avoid warnings from TensorFlow
tf.get_logger().setLevel('ERROR')

tf.random.set_seed(1) # Set global random seed for reproducibility
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
import mitsuba as mi

# Import Sionna RT components
from mysionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera

In [16]:
scene = load_scene("Indoor/indoor.xml")

In [17]:
#################配置发端天线阵列#################
scene.tx_array = PlanarArray(num_rows=2,
                             num_cols=2,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="tr38901",
                             polarization="cross")

#################配置收端天线阵列#################
scene.rx_array = PlanarArray(num_rows=2,
                             num_cols=2,
                             vertical_spacing=0.5,
                             horizontal_spacing=0.5,
                             pattern="dipole",
                             polarization="cross")

################创建发射机#########################
tx = Transmitter(name="tx1",
                 position=[0,0,2.95])
################ 将发射机加入到场景中##############
scene.add(tx)
tx = Transmitter(name="tx2",
                 position=[0,4.95,2.95])
scene.add(tx)
#################创建接收机########################
rx = Receiver(name="rx1",
              position=[2,2,1.5])
################ 将接收机加入到场景中##############
scene.add(rx)
rx = Receiver(name="rx2",
                position=[2.5,-4.5,1.3])
scene.add(rx)

In [18]:
scene.frequency = 2.14e9 # in Hz; implicitly updates RadioMaterials
scene.synthetic_array = True # If set to False, ray tracing will be done per antenna element (slower for large arrays)

In [19]:
paths = scene.compute_paths(max_depth=4,diffraction=True,scattering=False,edge_diffraction=True)

In [20]:
scene.preview(paths=paths)

Renderer(camera=PerspectiveCamera(aspect=1.31, children=(DirectionalLight(intensity=0.25, position=(0.0, 0.0, …

In [21]:
paths.a.shape

TensorShape([1, 2, 8, 2, 8, 132, 1])

In [22]:
paths.types.shape

TensorShape([1, 132])